In [16]:
#Assiociation rule mining

import re
import Orange
import pandas as pd
from pandas import DataFrame
import math
from Orange.data import Table

data = pd.read_csv(r'C:/Users/---/HR_comma_sep.csv') #import file
#transform file to orange dataframe

word = re.compile("\w+")

s = ''
for i in range(len(data)):
    s = s + str(data.left[i])+ ' ' +str(data.last_evaluation[i])+ ' ' +str(data.number_project[i])+ ' ' +str(data.average_montly_hours[i])+ ' '+str(data.time_spend_company[i])+ ' ' +str(data.Work_accident[i])+ ' ' +str(data.satisfaction_level[i])+ ' ' +str(data.salary[i])+'\n'

all_items = set(word.findall(s))
domain = Orange.data.Domain([])
domain.add_metas({Orange.orange.newmetaid(): Orange.feature.Continuous(n)
                  for n in all_items}, True)

data = Orange.data.Table(domain)
for e in s.splitlines():
    ex = Orange.data.Instance(domain)
    for m in re.findall("\w+", e):
        ex[m] = 1
    data.append(ex)
    

In [18]:
#create association rules with min support = 0.01(some case are rare but important with high lift)
rules = Orange.associate.AssociationRulesSparseInducer(data, support = 0.01)                             
all_confi = []
cosine_val = []
#store the measurement and rules
for i, r in enumerate(rules):
    all_confi.append(r.n_applies_both/max(r.n_applies_left, r.n_applies_right))
    cosine_val.append(r.n_applies_both/math.sqrt(r.n_applies_left*r.n_applies_right))
    
con_lift = pd.DataFrame(
    { "support":[r.support for r in rules ],
     "confidence" :[r.confidence for r in rules ],
     "lift" :[r.lift for r in rules ],
     "rule" :[r for r in rules ],
     "all_confi" :[i for i in all_confi],
     "cosine_val" :[i for i in cosine_val]

    })
#rearrange the sequence of the dataframe.    
con_lift = con_lift[['support', 'confidence', 'lift', 'all_confi', 'cosine_val', 'rule']]
#transform to dataframe to sort the value
con_lift = DataFrame(con_lift)   
con_lift = con_lift.sort_values(by=['all_confi'], ascending=[False])

In [19]:
con_lift.head()

,support,confidence,lift,all_confi,cosine_val,rule
5958,0.582239,1.000000,1.000000,0.582239,0.763046,3 -> 0
5959,0.582239,0.582239,1.000000,0.582239,0.763046,0 -> 3
5251,0.487766,0.487766,1.000000,0.487766,0.698402,0 -> low
5250,0.487766,1.000000,1.000000,0.487766,0.698402,low -> 0
4907,0.281819,0.484026,0.992333,0.484026,0.528827,3 -> low


In [20]:
con_lift.describe()

,support,confidence,lift,all_confi,cosine_val
count,6222.000000,6222.000000,6222.000000,6222.000000,6222.000000
mean,0.024163,0.326541,1.438664,0.057672,0.156521
std,0.035442,0.335261,1.199043,0.066418,0.074567
min,0.010067,0.010067,0.141580,0.010067,0.042903
25%,0.011467,0.033354,0.981210,0.026027,0.113823
50%,0.014001,0.182271,1.034814,0.033354,0.129792
75%,0.018068,0.517147,1.418811,0.060027,0.174126
max,0.582239,1.000000,13.623914,0.582239,0.763046


In [27]:
data.view()

AttributeError: 'Orange.data.Table' has no attribute 'view'

In [30]:
data[1]

['1', 0.86, '5', 262, 6, '0', 0.80, '0', 'sales', 'medium']